<a href="https://colab.research.google.com/github/Arush-Pimpalkar/GW_Project/blob/main/Copy_of_model_v5_cont.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/Continous_Check.zip -d /content

Streaming output truncated to the last 5000 lines.
  inflating: /content/Continous_Check/Data/SNR=9.12972451787368_m1=14_m2=16_16675.png  
  inflating: /content/Continous_Check/Data/SNR=15.474567452888909_m1=22_m2=25_15176.png  
  inflating: /content/Continous_Check/Data/SNR=12.21747781393573_m1=21_m2=27_18178.png  
  inflating: /content/Continous_Check/Data/SNR=9.036493239444658_m1=12_m2=15_18220.png  
  inflating: /content/Continous_Check/Data/SNR=8.798491311184563_m1=30_m2=13_18240.png  
  inflating: /content/Continous_Check/Data/SNR=17.43997473663319_m1=28_m2=24_15242.png  
  inflating: /content/Continous_Check/Data/SNR=8.336629090228255_m1=21_m2=23_18243.png  
  inflating: /content/Continous_Check/Data/SNR=5.613763639237537_m1=13_m2=18_12206.png  
  inflating: /content/Continous_Check/Data/SNR=8.670430191467078_m1=18_m2=16_12207.png  
  inflating: /content/Continous_Check/Data/SNR=13.049218409095388_m1=11_m2=20_13738.png  
  inflating: /content/Continous_Check/Data/SNR=8.473930469

In [3]:
from PIL import Image, UnidentifiedImageError
import os

def delete_corrupted_images(image_directory):
    for root, dirs, files in os.walk(image_directory):
        for file in files:
            if file.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif')):
                file_path = os.path.join(root, file)
                try:
                    img = Image.open(file_path)
                    img.verify()  # Verify that it is, in fact, an image
                except (IOError, SyntaxError, UnidentifiedImageError) as e:
                    print(f"Deleting corrupted file: {file_path} - {e}")
                    os.remove(file_path)

delete_corrupted_images('/content/Continous_Check/Data')


Deleting corrupted file: /content/Continous_Check/Data/SNR=10.211314771372491_m1=10_m2=18_37.png - cannot identify image file '/content/Continous_Check/Data/SNR=10.211314771372491_m1=10_m2=18_37.png'


In [4]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image size
img_size = (469, 369)

# Load CSV files
df_train = pd.read_csv(
    "/content/Continous_Check/cont_data_train.csv"
)
df_test = pd.read_csv(
    "/content/Continous_Check/cont_data_test.csv"
)
df_val = pd.read_csv(
    "/content/Continous_Check/cont_data_val.csv"
)

# Data generator for scaling images
scale = ImageDataGenerator(rescale=1.0 / 255)

# Create train, test, and validation generators
train_generator = scale.flow_from_dataframe(
    df_train,
    x_col="Path",
    y_col="Label",
    target_size=img_size,
    class_mode="raw",  # For regression, use 'raw'
    batch_size=32,
)

test_generator = scale.flow_from_dataframe(
    df_test,
    x_col="Path",
    y_col="Label",
    target_size=img_size,
    class_mode="raw",  # For regression, use 'raw'
    batch_size=32,
)

val_generator = scale.flow_from_dataframe(
    df_val,
    x_col="Path",
    y_col="Label",
    target_size=img_size,
    class_mode="raw",  # For regression, use 'raw'
    batch_size=32,
)

Found 29645 validated image filenames.
Found 3705 validated image filenames.
Found 3707 validated image filenames.


/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 1 invalid image filename(s) in x_col="Path". These filename(s) will be ignored.
  warnings.warn(


In [5]:


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the regression model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))  # Single neuron with linear activation for regression

# Compile the model
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)

# Print model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 467, 367, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 233, 183, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 231, 181, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 115, 90, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 113, 88, 128)      73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 56, 44, 128)       0

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=0.001
)

# Fit the model with callbacks
history = model.fit(
    train_generator,
    epochs=8,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/8
741/927 [======================>.......] - ETA: 1:44 - loss: 14.6298 - mean_absolute_error: 2.9285

In [ ]:
predictions = model.predict(test_generator)

In [ ]:
# Get the actual labels
actual_labels = df_test['Label'].values   # If using ImageDataGenerator

# Create a DataFrame for comparison
comparison_df = pd.DataFrame({
    'Actual': actual_labels,
    'Predicted': predictions.flatten() # Flatten the predictions if necessary
})

# Display the comparison
print(comparison_df)

In [ ]:
print(predictions)

In [ ]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(actual_labels, predictions.flatten())
print(mae)